In [1]:
import pandas as pd
df=pd.read_csv('quora_duplicate_questions.tsv',sep='\t')
df.dropna(inplace=True)
df = df.iloc[0:50000,:]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 0 to 49999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            50000 non-null  int64 
 1   qid1          50000 non-null  int64 
 2   qid2          50000 non-null  int64 
 3   question1     50000 non-null  object
 4   question2     50000 non-null  object
 5   is_duplicate  50000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 2.7+ MB


In [2]:
df.drop(columns=['id','qid1','qid2','is_duplicate'],inplace=True)

In [3]:
from transformers import pipeline
import tensorflow as tf
model = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')
positive_grad=['amusement','excitement','pride','optimism','gratitude','joy','admiration','love','approval','caring','desire','relief']
neutral_grad=['confusion','curiosity','realization','surprise','neutral']
negative_grad=['anger','disgust','grief','fear','sadness','nervousness','annoyance','disappointment','embarrassment','remorse','disapproval']
total_grad=positive_grad+neutral_grad+negative_grad

In [4]:

def predict_emotion(text):
    return model(text)[0]['label']


In [5]:
from tqdm import tqdm
for index, row in tqdm(df.iterrows(), total=len(df), desc='Processing rows'):
    input_text = row['question1']
    target_text = row['question2']

    input_emotion = predict_emotion(input_text)
    target_emotion = predict_emotion(target_text)

    if(input_emotion==target_emotion or input_emotion=='neutral' or target_emotion=='neutral'):
        df.drop(index, inplace=True)

    else:
        if(total_grad.index(input_emotion)>total_grad.index(target_emotion)):
            df.at[index,'question1'],df.at[index,'question2']=df.at[index,'question2'],df.at[index,'question1']
        emotion_transition = f"{input_emotion} to {target_emotion}: "
        modified_input_text = emotion_transition + input_text
        df.at[index, 'question1'] = modified_input_text

Processing rows: 100%|██████████| 50000/50000 [1:16:10<00:00, 10.94it/s]


In [9]:
df.to_csv('quora1',index=False)